# Objetivo: Desarrollo de funciones para procesar productos Sentinel-2 co snappy
# Fecha: 26/07/2024
# Autor: Gustavo V. Diaz

In [2]:
# Para manejo de raster


# Para abrir y bajar archivos en lista de bajada
import pandas as pd
import os
import sys
sys.path.append(r'../utils')
import mod_dloader as mdl

# Para manipular archivo bajado
import snappy
from snappy import Product
from snappy import ProductIO
from snappy import ProductUtils
from snappy import WKTReader
from snappy import HashMap
from snappy import GPF

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Lectura de df generado para seleccionar productos sentinel-2
path2df = r'/src/output/research_20240725T223256_Tratayen.xls'
df = pd.read_excel(path2df, sheet_name='resume_search')
display(df)
# Pude abrir el xls pero debo guardar la planilla con formato xls y no xlsx como lo había hecho.

**Objetivo para dirigir la misión** \
**Misión**: procesar cada producto S2 con la geometría en la misma proyección que el producto S2\
**Objetivo**: Bajar un producto para leer la proyección del mismo y así hacer la transformación al objeto vectorial

In [ ]:
# Id de ejemplo para obtener el tipo de proyección del producto
# Definiciones
user = 'gus838@gmail.com'
passw = 'Ul!RsPWTPuw3'
verbose = True
aux_key = 'access_token'
kc_token = 'KEYCLOAK_TOKEN'
prod_series = df.loc[0]
id_prod_series = prod_series.Id
name_prod_series = prod_series.Name
output_path = r'/src/output/'
str_token = mdl.get_keycloak(user, passw, verbose)
os.environ[kc_token] = str_token
# print(id_prod_series)

In [ ]:
# Bajo el producto para examinarlo con snappy



for row in df.iterrows():
    prod_id = row[1]['Id']
    prod_name = row[1]['Name']
    str_token = mdl.get_keycloak(user, passw, verbose)
    os.environ[kc_token] = str_token
    print('Variables para generadas en cada iteración:')
    print(f'Id Producto: {prod_id}',f'Nombre producto: {prod_name}',f'user: {user}',f'Key_cloak: {str_token}', sep='\n')
    print(f'Id Producto: {prod_id}',f'\nNombre producto: {prod_name}',f'\nuser: {user}')
    print()
    file2verif = os.path.join(output_path,prod_name + '.zip')
    if os.path.isfile(file2verif):
        print(f'Archivo {file2verif} existente')
        pass
    else:
        print(f'Archivo {file2verif} NO existente, bajando')
        mdl.prod_downloader_2(prod_id, os.environ[kc_token], output_path, prod_name, verbose)
    # BREAK de debug, solo permite que se baje un solo producto (el primero de la lista)
    break
#     if verbose:
#         print("Token de variable 'str_token'", str_token, sep='\n')
#         print('Variable de producto a ingresar a función "Bajadora"', prod_name, prod_id, sep='\n')

In [ ]:
# Manejo de producto sentinel para leer proyección de producto
product = ProductIO.readProduct(file2verif)

# Procesamiento de datos
width = product.getSceneRasterWidth()
print(f"Ancho: {width} px\n")
height = product.getSceneRasterHeight()
print(f"Alto: {height} px\n")
prod_name = product.getName()
print(f"Nombre: {prod_name}\n")
band_names = product.getBandNames()
# print(f"Nombre de bandas: {band_names}")
# print("Band names: {}".format(", ".join(band_names)))
str_band_n = ", ".join(band_names).split(", ")
print(f"Cantidad de bandas: {len(str_band_n)}\n")
# display(len(str_band_n), str_band_n)
crs_raster = product.getSceneCRS()
geocod_raster = product.getSceneGeoCoding()
# print("Muestras de proyección de producto", type(crs_raster), crs_raster, type(geocod_raster), geocod_raster, sep = '\n')

In [ ]:
# from osgeo import gdal -> No funciona, no encuentra el módulo _gdal
# !python3 test_script.py -> No funciona, ValueError: filedescriptor out of range in select()
# Intento por la vía del scripting en python llamando a un script ejecutable
# Ya logré obtener geometría de kml y reproyectar coordenadas de geometría a proyección que leí de producto Sentinel
# Ahora voy por levantar el archivo que tiene el WKT reproyectado y cortar el producto para verlo en pantalla.

## Archivo que contiene wkt reproyectado
wkt_path = r'./aux_files/wkt_reproj_file.txt'
## Archivo que contiene wkt original
wkt_orig_path = r'./aux_files/wkt_file.txt'

with open(wkt_path, 'r') as f:
    wkt_reproj = f.readline()

with open(wkt_orig_path, 'r') as f:
    wkt_orig = f.readline()

print("Lectura de archivo contenedor de WKT reproyectado y orignal", wkt_reproj, wkt_orig, sep='\n')

In [ ]:
## Lectura de WKT con snappy
geometry = WKTReader().read(wkt_reproj)
geometry_4326 = WKTReader().read(wkt_orig)
print(geometry)
print(geometry_4326)
# Aparentemente leyó correctemente la geometría definida en WKT

In [ ]:
# Objetivo 2 del día Cortar producto por geometría
SubsetOp = snappy.jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
geometry = WKTReader().read(wkt_reproj)
HashMap = snappy.jpy.get_type('java.util.HashMap')
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geometry_4326)
product_subset = GPF.createProduct('Subset', parameters, product)

In [ ]:
## Verificicación de creación de subset
width = product_subset.getSceneRasterWidth()
print("Width: {}px".format(width))
height = product_subset.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = product_subset.getBandNames()
band_list = ", ".join(band_names).split(', ')
display(band_list)
# print("Band names: {}".format(", ".join(band_names)))
band = product_subset.getBand(band_names[0])
print(band.getRasterSize())

In [ ]:
 def plotBand(product,band,vmin,vmax):
    band=product.getBand(band)
    w=band.getRasterWidth()
    h=band.getRasterHeight()
    print(w,h)
    band_data=np.zeros(w*h,np.float32)
    band.readPixels(0,0,w,h,band_data)
    band_data.shape=h,w
    width=12
    height=12
    plt.figure(figsize=(width,height))
    imgplot=plt.imshow(band_data,cmap=plt.cm.binary,vmin=vmin,vmax=vmax)

    return imgplot

In [ ]:
plotBand(product_subset, 'B2',0, 100)